# Data preprocessing

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# Dataset used for analysis: the 4th seasonal data of Airbnb listing
df_all = pd.read_csv('./data/listings.csv')

In [3]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6882 entries, 0 to 6881
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6882 non-null   int64  
 1   listing_url                                   6882 non-null   object 
 2   scrape_id                                     6882 non-null   int64  
 3   last_scraped                                  6882 non-null   object 
 4   source                                        6882 non-null   object 
 5   name                                          6882 non-null   object 
 6   description                                   0 non-null      float64
 7   neighborhood_overview                         5013 non-null   object 
 8   picture_url                                   6882 non-null   object 
 9   host_id                                       6882 non-null   i

## Features selection

* The ultimate goal of our project is to predict the price of Airbnb room for customers by giving information they care about when looking for a rental. In this way, features that customers will probably be interested in, including room type, accommodates, neighborhood, review ratings and so on, were selected for further analysis.
* As a result, all selected features are: neighborhood, room type, bathroom type, bedroom number, bathroom number, beds number, and review rating.
* As some of our selected features were not explicitly listed in the raw dataset, features such as beds number, bedroom number and bathroom type will be extracted from columns like `name` and `bathrooms_text`.

In [4]:
selected_columns = ['id', 'name', 'neighbourhood_cleansed','room_type','accommodates','bathrooms_text','bedrooms', 'beds','review_scores_rating', 'price'] 

In [5]:
df = df_all[selected_columns]
df = df.rename(columns={'neighbourhood_cleansed': 'neighbourhood', 'review_scores_rating': 'rating'})
df.head()

,id,name,neighbourhood,room_type,accommodates,bathrooms_text,bedrooms,beds,rating,price
0,6606,Guesthouse in Seattle · ★4.60 · 1 bedroom · 1 ...,Wallingford,Entire home/apt,1,1 bath,NaN,1.0,4.60,$99.00
1,9419,Rental unit in Seattle · ★4.72 · 1 bedroom · 1...,Georgetown,Private room,2,3 shared baths,NaN,1.0,4.72,$67.00
2,9531,Home in Seattle · ★4.97 · 2 bedrooms · 3 beds ...,Fairmount Park,Entire home/apt,4,1 bath,NaN,3.0,4.97,$184.00
3,9534,Guest suite in Seattle · ★4.99 · 2 bedrooms · ...,Fairmount Park,Entire home/apt,3,1 bath,NaN,2.0,4.99,$155.00
4,9596,Rental unit in Seattle · ★4.56 · 1 bedroom · 4...,Wallingford,Entire home/apt,4,1 bath,NaN,4.0,4.56,NaN


## Handling of missing values and feature exctracting

* We will give the prediction of `price` in this project, since 378 of 6882 rooms have missing `price` values, these records will be simply removed from further analysis.
* Primary analysis for all selected features are listed below:
    * neighborhood: explicitly included in the `neighbourhood` column
    * room type: explicitly included in the `room_type` column
    * bedroom number: implicitly included in the `name` column
    * beds number: implicitly included in the `name` column
    * bathroom type: implicitly included in the `bathrooms_text` column
    * bathroom number: implicitly included in the `bathrooms_text` column
    * review rating: some of rating scores were missing from `review_scores_rating` column, for these records, we will try to extract the rating value from `name` column

In [6]:
df.isna().sum()

id                   0
name                 0
neighbourhood        0
room_type            0
accommodates         0
bathrooms_text       0
bedrooms          6882
beds                56
rating            1070
price              378
dtype: int64

In [7]:
# NA records in `price` will be removed
df = df.loc[df["price"].notnull(), :]

In [8]:
# some information are implicitly included within the `name` column 
df.iloc[1]['name']

'Rental unit in Seattle · ★4.72 · 1 bedroom · 1 bed · 3 shared baths'

* Handling of `rating`, `bed number` and `bedroom number`

In [9]:
rating_ptn = r'★\s*(\d+\.\d+)'
bedroom_ptn = r'(\d+)\s+bedroom'
bed_ptn = r'(\d+)\s+bed'


def fillna_rating(idx, row):
    '''
    fill rating with information extracted from `name` if `review_scores_rating` is na
    '''
    if pd.isna(row['rating']):
        rating = re.findall(rating_ptn, row['name'])
        df.loc[idx, 'rating'] = float(rating[0]) if rating else -1


def fillna_bedroom(idx, row):
    '''
    fill bedroom with information extracted from `name` if `bedrooms` is na
    '''
    if pd.isna(row['bedrooms']):
        if 'studio' in row['name'].lower():
            df.loc[idx, 'bedrooms'] = 0
        else:
            bedrooms = re.findall(bedroom_ptn, row['name'])
            df.loc[idx, 'bedrooms'] = int(bedrooms[0]) if bedrooms else -1


def fillna_bed(idx, row):
    '''
    fill beds with information extracted from `name` if `beds` is na
    '''
    if pd.isna(row['beds']):
        beds = re.findall(bed_ptn, row['name'])
        df.loc[idx, 'beds'] = int(beds[0]) if beds else -1    


for idx, row in df.iterrows():
    fillna_rating(idx, row)
    fillna_bedroom(idx, row)
    fillna_bed(idx, row)

In [10]:
df.head()

,id,name,neighbourhood,room_type,accommodates,bathrooms_text,bedrooms,beds,rating,price
0,6606,Guesthouse in Seattle · ★4.60 · 1 bedroom · 1 ...,Wallingford,Entire home/apt,1,1 bath,1.0,1.0,4.60,$99.00
1,9419,Rental unit in Seattle · ★4.72 · 1 bedroom · 1...,Georgetown,Private room,2,3 shared baths,1.0,1.0,4.72,$67.00
2,9531,Home in Seattle · ★4.97 · 2 bedrooms · 3 beds ...,Fairmount Park,Entire home/apt,4,1 bath,2.0,3.0,4.97,$184.00
3,9534,Guest suite in Seattle · ★4.99 · 2 bedrooms · ...,Fairmount Park,Entire home/apt,3,1 bath,2.0,2.0,4.99,$155.00
5,9909,Home in Seattle · ★4.80 · 2 bedrooms · 2 beds ...,Fairmount Park,Entire home/apt,4,1 bath,2.0,2.0,4.80,$228.00


* Handling of `bathroom number` and `bath type`

In [11]:
df['bathrooms_text'].unique()

array(['1 bath', '3 shared baths', '1.5 baths', '4 shared baths',
       '1 shared bath', '1 private bath', '2 baths', '3 baths', '4 baths',
       '2.5 baths', '1.5 shared baths', 'Shared half-bath',
       '2 shared baths', '3.5 baths', 'Half-bath', '2.5 shared baths',
       '3.5 shared baths', '0 shared baths', '4.5 baths',
       '16 shared baths', '0 baths', '6 baths', 'Private half-bath',
       '6.5 baths', '7 baths', '5.5 baths', '5 baths'], dtype=object)

* Bathroom numbers and types are primarily extracted from `bathrooms_text` column.

In [12]:
bath_txt_ptn = r'(\d+\.?\d*)'

def fill_bath(row):
    bath = re.findall(bath_txt_ptn, row)
    return float(bath[0]) if bath else -1


def fill_bath_type(text):
    if 'shared' in text:
        return 'shared'
    elif 'private' in text:
        return 'private'
    elif 'half-bath' in text:
        return 'half-bath'
    else:
        return 'standard'


df['bathrooms'] = df['bathrooms_text'].apply(fill_bath)
df['bath_type'] = df['bathrooms_text'].apply(fill_bath_type) 

In [13]:
df.head()

,id,name,neighbourhood,room_type,accommodates,bathrooms_text,bedrooms,beds,rating,price,bathrooms,bath_type
0,6606,Guesthouse in Seattle · ★4.60 · 1 bedroom · 1 ...,Wallingford,Entire home/apt,1,1 bath,1.0,1.0,4.60,$99.00,1.0,standard
1,9419,Rental unit in Seattle · ★4.72 · 1 bedroom · 1...,Georgetown,Private room,2,3 shared baths,1.0,1.0,4.72,$67.00,3.0,shared
2,9531,Home in Seattle · ★4.97 · 2 bedrooms · 3 beds ...,Fairmount Park,Entire home/apt,4,1 bath,2.0,3.0,4.97,$184.00,1.0,standard
3,9534,Guest suite in Seattle · ★4.99 · 2 bedrooms · ...,Fairmount Park,Entire home/apt,3,1 bath,2.0,2.0,4.99,$155.00,1.0,standard
5,9909,Home in Seattle · ★4.80 · 2 bedrooms · 2 beds ...,Fairmount Park,Entire home/apt,4,1 bath,2.0,2.0,4.80,$228.00,1.0,standard


In [14]:
# NA values are now marked as -1

df = df.loc[(df['rating'] != -1) 
            & (df['bedrooms'] != -1) 
            & (df['beds'] != -1)
            & (df['bathrooms'] != -1), :]

In [15]:
df.isna().sum()

id                0
name              0
neighbourhood     0
room_type         0
accommodates      0
bathrooms_text    0
bedrooms          0
beds              0
rating            0
price             0
bathrooms         0
bath_type         0
dtype: int64

In [16]:
df.shape

(5475, 12)

In [17]:
df.describe()

,id,accommodates,bedrooms,beds,rating,bathrooms
count,5.475000e+03,5475.000000,5475.000000,5475.000000,5475.000000,5475.000000
mean,3.416739e+17,4.180457,1.661553,2.237443,4.795353,1.435251
std,4.063029e+17,2.600543,1.186012,1.504671,0.356755,0.811386
min,6.606000e+03,1.000000,0.000000,1.000000,1.000000,0.000000
25%,2.529550e+07,2.000000,1.000000,1.000000,4.760000,1.000000
50%,5.163902e+07,4.000000,1.000000,2.000000,4.890000,1.000000
75%,7.538532e+17,6.000000,2.000000,3.000000,4.980000,2.000000
max,1.041325e+18,16.000000,13.000000,16.000000,5.000000,16.000000


In [18]:
columns_to_keep = ['accommodates', 'bedrooms', 'beds', 'bathrooms', 'rating',
                  'neighbourhood', 'room_type', 'bath_type', 'price']
df = df[columns_to_keep]
df.head()

,accommodates,bedrooms,beds,bathrooms,rating,neighbourhood,room_type,bath_type,price
0,1,1.0,1.0,1.0,4.60,Wallingford,Entire home/apt,standard,$99.00
1,2,1.0,1.0,3.0,4.72,Georgetown,Private room,shared,$67.00
2,4,2.0,3.0,1.0,4.97,Fairmount Park,Entire home/apt,standard,$184.00
3,3,2.0,2.0,1.0,4.99,Fairmount Park,Entire home/apt,standard,$155.00
5,4,2.0,2.0,1.0,4.80,Fairmount Park,Entire home/apt,standard,$228.00


## Data type and encoding

* `price` has to be float type rather than string
* Encoding categorical features like `neighbourhood` in one-hot pattern

In [19]:
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5475 entries, 0 to 6814
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   accommodates   5475 non-null   int64  
 1   bedrooms       5475 non-null   float64
 2   beds           5475 non-null   float64
 3   bathrooms      5475 non-null   float64
 4   rating         5475 non-null   float64
 5   neighbourhood  5475 non-null   object 
 6   room_type      5475 non-null   object 
 7   bath_type      5475 non-null   object 
 8   price          5475 non-null   float64
dtypes: float64(5), int64(1), object(3)
memory usage: 427.7+ KB


In [21]:
df_prim = df[df['price'] <= 4000]
df_prim.to_csv('./data/listing_primary.csv', index=False)

* Features including `neighbourhood`, `room_type` and `bath_type` are categorical data, we thus convert these values into one-hot encoding pattern.

In [22]:
df['neighbourhood'].unique()

array(['Wallingford', 'Georgetown', 'Fairmount Park', 'Whittier Heights',
       'Sunset Hill', 'Eastlake', 'Fremont', 'Green Lake', 'Portage Bay',
       'Phinney Ridge', 'Crown Hill', 'Columbia City', 'Lawton Park',
       'North Queen Anne', 'West Queen Anne', 'First Hill', 'Broadway',
       'North Admiral', 'International District', 'North Beacon Hill',
       'Greenwood', 'Cedar Park', 'Mount Baker', 'Mann', 'Ravenna',
       'Belltown', 'University District', 'Harrison/Denny-Blaine',
       'South Delridge', 'Broadview', 'Maple Leaf', 'East Queen Anne',
       'Atlantic', 'Highland Park', 'West Woodland', 'Laurelhurst',
       'Haller Lake', 'Madison Park', 'Fauntleroy', 'Madrona',
       'Loyal Heights', 'Gatewood', 'Leschi', 'Westlake', 'Adams',
       'North Delridge', 'Bryant', 'Seward Park', 'Stevens',
       'Central Business District', 'Pioneer Square', 'Pike-Market',
       'Yesler Terrace', 'Alki', 'Bitter Lake', 'Lower Queen Anne',
       'Windermere', 'Minor', 'Rainie

In [23]:
df['bath_type'].unique()

array(['standard', 'shared', 'private'], dtype=object)

In [24]:
df['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [25]:
df['room_type'] = df['room_type'].map({'Entire home/apt': 'entire', 
                                      'Private room': 'private', 
                                      'Shared room': 'shared', })

In [26]:
df_clean = pd.get_dummies(df, columns=['bath_type', 'room_type', 'neighbourhood'])
df_clean.head()

,accommodates,bedrooms,beds,bathrooms,rating,price,bath_type_private,bath_type_shared,bath_type_standard,room_type_entire,...,neighbourhood_Victory Heights,neighbourhood_View Ridge,neighbourhood_Wallingford,neighbourhood_Wedgwood,neighbourhood_West Queen Anne,neighbourhood_West Woodland,neighbourhood_Westlake,neighbourhood_Whittier Heights,neighbourhood_Windermere,neighbourhood_Yesler Terrace
0,1,1.0,1.0,1.0,4.60,99.0,False,False,True,True,...,False,False,True,False,False,False,False,False,False,False
1,2,1.0,1.0,3.0,4.72,67.0,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,4,2.0,3.0,1.0,4.97,184.0,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
3,3,2.0,2.0,1.0,4.99,155.0,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
5,4,2.0,2.0,1.0,4.80,228.0,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False


* Cleaned data (df_clean) is dumped into disk for training out model

In [27]:
df_clean = df_clean[df_clean['price'] <= 4000]
df_clean.to_csv('./data/listing_cleaned.csv', index=False)

In [28]:
# dump a df with only one row to generate user input in Streamlit app
df_input = df_clean.iloc[0:1]
df_input = df_input.drop('price', axis=1)
df_input.to_csv('./data/default_input.csv', index=False)